## 第八章 集成学习算法（Boosting和Bagging）

　　集成学习(ensemble learning)可以说是现在非常火爆的机器学习方法了。它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务。也就是我们常说的“博采众长”。集成学习可以用于分类问题集成，回归问题集成，特征选取集成，异常点检测集成等等，可以说所有的机器学习领域都可以看到集成学习的身影。  
　　如何产生并结合‘好而不同’的个体学习器，恰是集成学习研究的核心，如果个体学习器之间没有差异性，结果就是“三个臭皮匠。还是一个臭皮匠”，也就是合成的学习器基本不会有能力提升。  
　　按照个体学习器之间的关系，分为Bagging、Boosting、Stacking三大类。  
* https://www.jiqizhixin.com/articles/2018-12-28-11 很好
* https://blog.csdn.net/m0_37712157/article/details/81142217  
* https://blog.csdn.net/chkay399/article/details/82117832  
* https://blog.csdn.net/yuanliang861/article/details/85012734  

### 一、随机森林算法
* https://blog.csdn.net/weixin_41940752/article/details/98717868 
* https://blog.csdn.net/Elaine1006/article/details/85126926 (非常好）
* https://blog.csdn.net/yangyin007/article/details/82385967

#### 1.算法介绍

　　Bagging的原理首先是基于自助采样法（bootstrap sampling）随机得到一些样本集训练，用来分别训练不同的基学习器，然后对不同的基学习器得到的结果投票得出最终的分类结果。自助采样法得到的样本大概会有63%的数据样本被使用，剩下的可以用来做验证集。  
　　随机森林（Random Forest）其实也算Bagging的一种，但是有一点区别是随机森林在构建决策树的时候，除了随机选择样本，还会随机选择一部分特征来进行划分。由于随机森林的二重随机性，它具有良好的学习性能。   
　　随机森林对多元共线性不敏感，结果对缺失数据和非平衡的数据比较稳健，可以很好地预测多达几千个解释变量的作用（Breiman 2001b），被誉为当前最好的算法之一（Iverson et al. 2008）。  
　　随机森林是一个比较优秀的模型，在我的项目的使用效果上来看，它对于多维特征的数据集分类有很高的效率，还可以做特征重要性的选择。运行效率和准确率较高，实现起来也比较简单。但是在数据噪音比较大的情况下会过拟合，过拟合的缺点对于随机森林来说还是较为致命的。  
* 1.具体算法  
（1）从样本集中通过重采样的方式产生n个样本  
（2）假设样本特征数目为a，对n个样本选择a中的k个特征，用建立决策树的方式获得最佳分割点  
（3）重复m次，产生m棵决策树  
（4）多数投票机制来进行预测  
* 2.随机森林优缺点  
优点：  
（1）它能够处理很高维度（feature很多）的数据，并且不用做特征选择（特征子集是随机选择的，子集大小可以控制）  
（2）训练完后，它能够给出feature重要大小  
（3）训练速度快，容易做成并行化方法。原因：训练时树与树之间是相互独立的  
（4）如果有很大一部分的特征遗失，仍可以维持准确度。  
（5）在数据集上表现良好，两个随机性的引入，使得随机森林不容易陷入过拟合，还具有很好的抗噪声能力。    
（6）随机森林算法中包含了对输入数据的重复自抽样过程，即所谓的bootstrap抽样。这样一来，数据集中大约三分之一将没有用于模型的训练而是用于测试  
（7）随机森林是一种多功能的机器学习算法，能够执行回归和分类的任务。   
缺陷：     
（1）耗时间长，CPU资源占用多    

#### 2.实例１：利用随机森林算法进行糖尿病预测
* https://www.cnblogs.com/xxtalhr/p/10859517.html  (非常好的对该数据集的探索性分析和数据分析)


　　该数据集最初来自国家糖尿病/消化/肾脏疾病研究所。数据集共有768个样本数据，每条数据有8个特征，一个label。数据的内容是皮马人的医疗记录，以及过去5年内是否有糖尿病。数据集的目标是基于数据集中包含的某些诊断测量来诊断性的预测患者是否患有糖尿病。  
* Pregnancies：怀孕次数   
* Glucose：血糖 
* BloodPressure：血压 (mm Hg) 
* SkinThickness：皮层厚度 (mm) 
* Insulin：胰岛素 2小时血清胰岛素（mu U / ml 
* BMI：体重指数 （体重/身高）^2 
* DiabetesPedigreeFunction：糖尿病谱系功能 
* Age：年龄 （岁） 
* Outcome：类标变量 （0或1）

In [1]:
#不出现警告信息
#import warnings
#warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  #导入数据处理模块中的标准化函数

#１.导入数据及生成特征数据和标签数据
data = pd.read_csv('./data_picture/chapter8/diabetes.csv')  #糖尿病数据
X = data.drop('Outcome',axis=1)
y = data['Outcome']

#生成训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

#2.数据标准化
SS = StandardScaler()         #生成实体类模块
scaler=SS.fit(X_train)        #本质上就是求各列均值和方差
X_train=scaler.transform(X_train)     #对数据各列进行标准化
X_test=scaler.transform(X_test) 

In [4]:
#3.利用参数遍历功能选择最好的训练模型
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

# 生成实例
pr_model=RandomForestClassifier()

#模型调参训练
n_estimators=range(95,110)   #树的个数遍历
max_depth= [4,5,6,7,8]   #给出学习率参数的遍历
par_grid = {'max_depth': max_depth,'n_estimators':n_estimators}   #给出学习率和树的个数遍历器
cv = ShuffleSplit(n_splits=10, test_size=0.3)
grid_search= GridSearchCV(pr_model, param_grid=par_grid,cv=cv)  #对模型遍历交叉验证生成器
grid_search.fit(X_train,y_train)
print('best_params=',grid_search.best_params_)
print('Best score:',grid_search.best_score_)
model=grid_search.best_estimator_

best_params= {'max_depth': 4, 'n_estimators': 104}
Best score: 0.7647398843930636


In [6]:
#4.模型评估
from sklearn.metrics import classification_report
print("训练集的模型评估指标：")
model_score=model.score(X_train,y_train)
print()
print('The accuracy of train data',model_score)
print('--------------------------------------------------------------------------')
y_train_predict=model.predict(X_train)
model_report1=classification_report(y_train,y_train_predict)
print(model_report1)
print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

print("测试集的模型评估指标：")
model_score=model.score(X_test,y_test)
print()
print('The accuracy of test data is',model_score)
print('--------------------------------------------------------------------------')
y_predict=model.predict(X_test)
model_report=classification_report(y_test,y_predict)
print(model_report)
print('--------------------------------------------------------------------------')

训练集的模型评估指标：

The accuracy of train data 0.8368055555555556
--------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      0.94      0.88       378
           1       0.85      0.64      0.73       198

    accuracy                           0.84       576
   macro avg       0.84      0.79      0.81       576
weighted avg       0.84      0.84      0.83       576

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
测试集的模型评估指标：

The accuracy of test data is 0.7604166666666666
--------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.91      0.83       122
           1       0.76      0.50      0.60        70

    accuracy                           0.76       192
   macro avg       0.76      0.70      0.72       192
weighted avg       0.76      0.76      0.75       192

--

### 二、Xgboost算法(分类和回归都很好)
* 事先需要下载安装这个包：pip install xgboost
* https://www.cnblogs.com/zongfa/p/9324684.html     最好的介绍了原理和参数解析，非常非常好，  
* https://www.toutiao.com/a6607591252282049028/     也不错，对Xgboost和GBDT在数学算法不同上的讲解比较透  
* https://blog.csdn.net/weixin_39750084/article/details/83154132     （很简洁的说明,非常好）
* https://blog.csdn.net/qunnie_yi/article/details/80129857    (揭秘Kaggle神器xgboost)

#### 1.Xgboost(eXtreme Gradient Boosting)介绍
　　XGBoost号称“比赛夺冠的必备大杀器”，横扫机器学习竞赛罕逢敌手。最近甚至有一位大数据/机器学习主管被XGBoost在项目中的表现惊艳到，盛赞其为“机器学习算法中的新女王”！在 Kaggle 的很多比赛中，我们可以看到很多 winner 喜欢用 xgboost，而且获得非常好的表现。xgboost是由华盛顿大学的陈天奇博士提出。　　  
　　XGBoost全名叫（eXtreme Gradient Boosting）极端梯度提升，经常被用在一些比赛中，其效果显著。它是大规模并行boosted tree的工具，它是目前最快最好的开源boosted tree工具包。XGBoost 所应用的算法就是 GBDT（gradient boosting decision tree）的改进，既可以用于分类也可以用于回归问题中，XGBoost在绝大多数的回归和分类问题上表现的十分顶尖。调参过程比较麻烦。  
　　XGBoost的原理在本质上与GBDT相同，GBDT算法只利用了一阶的导数信息，xgboost对损失函数做了二阶的泰勒展开，并在目标函数之外加入了正则项对整体求最优解，用以权衡目标函数的下降和模型的复杂程度，避免过拟合。  
　　GBDT在函数空间中利用梯度下降法进行优化,XGBoost在函数空间中用牛顿法进行优化。XGBoost与GBDT的比较优势其实就是牛顿法与梯度下降法的比较优势，梯度下降法用目标函数的一阶偏导、以负梯度方向作为搜索方向，只考虑目标函数在迭代点的局部性质，搜寻路线往往曲折往返速度慢，牛顿法在二阶导数的作用下，从函数的凸性出发，直接搜索怎样到达极值点，也就是说在选择方向时，不仅考虑当前坡度是否够大，还会考虑走了一步之后，坡度是否会变得更大。梯度下降是线性收敛，牛顿法是超线性的，至少二阶收敛。总之，牛顿法要比梯度下降法更具有全局判断能力。  
　　从集成算法的原理可以看出，Boosting过程是串行的结构，并不能并行。注意XGBoost的并行不是tree粒度的并行，XGBoost也是一次迭代完才能进行下一次迭代的（第t次迭代的代价函数里包含了前面t-1次迭代的预测值），XGBoost的并行是在特征粒度上的。  

* 注：GBDT在函数空间中利用梯度下降法进行优化,梯度下降法用目标函数的一阶偏导、以负梯度方向作为搜索方向。XGBoost在函数空间中用牛顿法进行优化，牛顿法在二阶导数的作用下，从函数的凸性出发，直接搜索怎样到达极值点，而不是构造搜索方向。

#### 2.Xgboost算法原理

![caption](./data_picture/chapter8/xgb1.png)
![caption](./data_picture/chapter8/xgb2.png)
![caption](./data_picture/chapter8/xgb3.png)
![caption](./data_picture/chapter8/xgb4.png)
![caption](./data_picture/chapter8/xgb5.png)

#### 3.Xgboost的优点

* 传统的GBDT以CART作为基分类器，XGboost 还支持线性分类器，这时候xgboost 相当于带L1 和L2 正则化项的逻辑斯蒂回归(分类问题) 或者线性回归。  
* 传统的GBDT在优化时只用到了一阶导数信息，xgboost 则对代价函数进行了二阶泰勒展开，同时用到了一阶和二阶导数，寻优速度更快。xgboost 还支持自定义代价函数，只要函数可一阶和二阶求导。  
* Xgboost 在代价函数中加入了正则项，用于控制模型的复杂度，正则项里包含了 树的叶子节点个数，每个叶子节点上输出的score 的L2 模的平方和。从Bias -variance  tradeoff 角度来讲，正则项降低了模型的variance，使学习出来的模型更加简单，防止过拟合，这也就是xgboost 优于传统CBDT的一个特性。  
* Shrinkage ,相当于学习速率(xgboost 中的eta) .Xgboost 在进行完一次迭代后，会将叶子节点上权重·乘上该系数，主要是为了削弱每棵树的影响，让后面有更大的学习空间，实际应用中，一般把eta 设置的小一点，然后迭代次数设置的大一点。  
* 列抽样，xgboost借鉴了随机森林的做法，支持列抽样，不仅能降低过拟合，还能减少计算，这也是xgboost 异于传统gdbt 的一个特性。  
* 缺失值的处理，对特征的值有缺失的样本，xgboost 可以自动学习出它分裂的方向。  
* xgboost 支持并行，不是在trees 粒度的并行，而是在特征粒度上的，决策树的学习最耗时的一个步骤就是对特征的值进行排序(因为要确定最佳分割点) ,xgboost 在训练之前，预先对数据进行了排序，然后保存了block 结构，后面的迭代中重复使用了这个结构，大大减少了计算量。在进行节点分裂时，需要计算每个特征的增益，最终选增益最大的那个特征去做分裂，那么各个特征的增益计算可以开多线程进行。  
* 可并行的近似直方图算法，树节点在进行分裂时，我们需要计算每个特征的每个分割点对应的增益，即用贪心法枚举所有节能的分割点，当数据无法一次载入内存或者在分布式情况下，贪心算法效率就会变得很低，所以xgboost 还提出了一种可并行的近似直方图算法，用于高效的生成候选的分割点。

#### 4.Xgboost调参
* https://blog.csdn.net/ruding/article/details/78328835  
* https://blog.csdn.net/wzmsltw/article/details/50994481

* max_depth：树的最大深度。缺省值为6，越大越容易过拟合通常取值：3-10。  
* min_child_weight：孩子节点中最小的样本权重和。缺省值为0,调大这个参数能够控制过拟合。  
* gamma : 缺省值为0,初始值通常设置在0.1-0.2范围内. 模型在默认情况下，对于一个节点的划分只有在其loss function 得到结果大于0的情况下才进行，而gamma 给定了所需的最低loss function的值.  

参考资料2中，Aarshay分享了他的调参经验，核心的思想是先调节影响最大的参数，再调节次要的参数。  

* 设置较大的learning_rate以及相应的最优n_estimator  
* 调节Tree-Specific Parameters  
* 调节max_depth以及min_samples_split  
* 调节min_samples_leaf  
* 调节max_features  
* 降低learning_rate，并相应地增加n_estimator  
* 参考GB的调参过程。对于XGBoost, max_depth、min_child_weight以及gamma是影响基本树预测准确率最为重要的三个因素  

#### 5.实例：利用Xgboost算法进行糖尿病预测
　　该数据集最初来自国家糖尿病/消化/肾脏疾病研究所。数据集共有768个样本数据，每条数据有8个特征，一个label。数据的内容是皮马人的医疗记录，以及过去5年内是否有糖尿病。数据集的目标是基于数据集中包含的某些诊断测量来诊断性的预测患者是否患有糖尿病。  
* Pregnancies：怀孕次数   
* Glucose：血糖 
* BloodPressure：血压 (mm Hg) 
* SkinThickness：皮层厚度 (mm) 
* Insulin：胰岛素 2小时血清胰岛素（mu U / ml 
* BMI：体重指数 （体重/身高）^2 
* DiabetesPedigreeFunction：糖尿病谱系功能 
* Age：年龄 （岁） 
* Outcome：类标变量 （0或1）

In [7]:
#不出现警告信息
#import warnings
#warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  #导入数据处理模块中的标准化函数

#１.导入数据及生成特征数据和标签数据
data = pd.read_csv('./data_picture/chapter8/diabetes.csv')  #糖尿病数据
X = data.drop('Outcome',axis=1)
y = data['Outcome']

#生成训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

#2.数据标准化
SS = StandardScaler()         #生成实体类模块
scaler=SS.fit(X_train)        #本质上就是求各列均值和方差
X_train=scaler.transform(X_train)     #对数据各列进行标准化
X_test=scaler.transform(X_test) 

In [8]:
#3.利用参数遍历功能选择最好的训练模型
from xgboost import XGBClassifier,XGBRegressor   #pip install xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

# 生成实例
pr_model=XGBClassifier()

#模型调参训练
#min_child_weight=[0,0.1,0.2,0.3,0.4,0.5]
#gamma=[0.1,0.12,0.14,0.16,0.18,0.20]
max_depth=range(5,10)
n_estimators=range(5,15)   #树的个数遍历
learning_rate= [0.01,0.1,0.3,0.5,0.8,1.0,1.2,1.4]   #给出学习率参数的遍历
par_grid = {'learning_rate': learning_rate,'n_estimators':n_estimators,'max_depth':max_depth}   #给出学习率和树的个数遍历器
#par_grid = {'min_child_weight': min_child_weight,'gamma':gamma,'max_depth':max_depth,'n_estimators':n_estimators}
cv = ShuffleSplit(n_splits=10, test_size=0.3)
grid_search= GridSearchCV(pr_model, param_grid=par_grid,cv=cv)  #对模型遍历交叉验证生成器
grid_search.fit(X_train,y_train)
print('best_params=',grid_search.best_params_)
print('Best score:',grid_search.best_score_)
model=grid_search.best_estimator_


best_params= {'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 9}
Best score: 0.7670520231213873


In [11]:
#3.利用参数遍历功能选择最好的训练模型
from xgboost import XGBClassifier,XGBRegressor   #pip install xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

# 生成实例
model=XGBClassifier()
model.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [12]:
#4.模型评估
from sklearn.metrics import classification_report
print("训练集的模型评估指标：")
model_score=model.score(X_train,y_train)
print()
print('The accuracy of train data',model_score)
print('--------------------------------------------------------------------------')
y_train_predict=model.predict(X_train)
model_report1=classification_report(y_train,y_train_predict)
print(model_report1)
print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

print("测试集的模型评估指标：")
model_score=model.score(X_test,y_test)
print()
print('The accuracy of test data is',model_score)
print('--------------------------------------------------------------------------')
y_predict=model.predict(X_test)
model_report=classification_report(y_test,y_predict)
print(model_report)
print('--------------------------------------------------------------------------')

训练集的模型评估指标：

The accuracy of train data 1.0
--------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       378
           1       1.00      1.00      1.00       198

    accuracy                           1.00       576
   macro avg       1.00      1.00      1.00       576
weighted avg       1.00      1.00      1.00       576

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
测试集的模型评估指标：

The accuracy of test data is 0.71875
--------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.77      0.80      0.78       122
           1       0.62      0.59      0.60        70

    accuracy                           0.72       192
   macro avg       0.70      0.69      0.69       192
weighted avg       0.72      0.72      0.72       192

----------------------------